# Model Evaluation - Working Version
This notebook evaluates the trained model and logs metrics to MLflow

In [4]:
import os

In [5]:
# Change to project root directory
os.chdir('c:\\Users\\deepu\\OneDrive\\Desktop\\ML_Pipeline')
%pwd

'c:\\Users\\deepu\\OneDrive\\Desktop\\ML_Pipeline'

In [6]:
# Initialize DagsHub for MLflow tracking
import dagshub
import mlflow

dagshub.init(repo_owner='DeepuHemant', repo_name='DataScience_Complete_ML_Pipeline', mlflow=True)

# Set credentials
os.environ['MLFLOW_TRACKING_URI'] = 'https://dagshub.com/DeepuHemant/DataScience_Complete_ML_Pipeline.mlflow'
os.environ['MLFLOW_TRACKING_USERNAME'] = 'DeepuHemant'
os.environ['MLFLOW_TRACKING_PASSWORD'] = '65e2450193707a8901e9b601e765263b90687496'

Accessing as DeepuHemant

Initialized MLflow to track repo "DeepuHemant/DataScience_Complete_ML_Pipeline"

Repository DeepuHemant/DataScience_Complete_ML_Pipeline initialized!

In [7]:
# Define configuration dataclass
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [8]:
# Import utilities
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories, save_json

In [9]:
# Configuration Manager
class ConfigurationManager:
    def __init__(self,
                 config_file_path=CONFIG_FILE_PATH,
                 params_file_path=PARAMS_FILE_PATH,
                 schema_file_path=SCHEMA_FILE_PATH):
        
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)
        
        create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_name=config.metric_file_name,
            target_column=schema.name,
            mlflow_uri=config.mlflow_uri
        )
        return model_evaluation_config

In [10]:
# Import required libraries
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import numpy as np
import joblib

In [11]:
# Model Evaluation Class - FIXED VERSION
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):
        # Load test data and model
        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        # Prepare test data
        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        # Set MLflow tracking URI
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        
        # Start MLflow run
        with mlflow.start_run():
            # Make predictions
            predicted_qualities = model.predict(test_x)

            # Calculate metrics
            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)

            # Save metrics locally
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            # Log parameters and metrics to MLflow
            mlflow.log_params(self.config.all_params)
            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)
            
            # ✅ MODEL LOGGING REMOVED - DagsHub doesn't support this endpoint
            # The model is already saved locally in artifacts/model_trainer/
            
            print("\n" + "="*60)
            print("✅ MODEL EVALUATION COMPLETED SUCCESSFULLY!")
            print("="*60)
            print(f"📊 Metrics:")
            print(f"   RMSE: {rmse:.4f}")
            print(f"   MAE:  {mae:.4f}")
            print(f"   R2:   {r2:.4f}")
            print("="*60)
            print(f"💾 Metrics saved to: {self.config.metric_file_name}")
            print(f"🔗 View MLflow run at DagsHub")
            print("="*60)

In [12]:
# Execute Model Evaluation Pipeline
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    model_evaluation.log_into_mlflow()
except Exception as e:
    print(f"❌ Error: {e}")
    raise e

[2025-11-25 20:14:06,575:INFO:common:yaml file: config\config.yaml loaded successfully]
[2025-11-25 20:14:06,597:INFO:common:yaml file: params.yaml loaded successfully]
[2025-11-25 20:14:06,625:INFO:common:yaml file: schema.yaml loaded successfully]
[2025-11-25 20:14:06,629:INFO:common:created directory at: artifacts]
[2025-11-25 20:14:06,631:INFO:common:created directory at: artifacts/model_evaluation]
[2025-11-25 20:14:08,582:INFO:common:json file saved at : artifacts\model_evaluation\metrics.json]

✅ MODEL EVALUATION COMPLETED SUCCESSFULLY!
📊 Metrics:
   RMSE: 0.6695
   MAE:  0.5263
   R2:   0.2452
💾 Metrics saved to: artifacts/model_evaluation/metrics.json
🔗 View MLflow run at DagsHub
🏃 View run gentle-sheep-221 at: https://dagshub.com/DeepuHemant/DataScience_Complete_ML_Pipeline.mlflow/#/experiments/0/runs/daaa6b40cd6746fe94b99b64d8f24da0
🧪 View experiment at: https://dagshub.com/DeepuHemant/DataScience_Complete_ML_Pipeline.mlflow/#/experiments/0
